Questions answering with embeddings.  Adapted from [OpenAI
Notebook](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb).

In [1]:
import datasets
import numpy as np

In [2]:
from minichain import EmbeddingPrompt, TemplatePrompt, show_log, start_chain

Load data with embeddings (computed beforehand)

In [3]:
gatsby = datasets.load_from_disk("gatsby")
gatsby.add_faiss_index("embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['passages', 'embeddings'],
    num_rows: 52
})

Fast KNN retieval prompt

In [4]:
class KNNPrompt(EmbeddingPrompt):
    def find(self, out, inp):
        res = gatsby.get_nearest_examples("embeddings", np.array(out), 1)
        return {"question": inp, "docs": res.examples["passages"]}

QA prompt to ask question with examples

In [5]:
class QAPrompt(TemplatePrompt):
    template_file = "gatsby.pmpt.tpl"

In [6]:
with start_chain("gatsby") as backend:
    question = "What did Gatsby do before he met Daisy?"
    prompt = KNNPrompt(backend.HuggingFaceEmbed("sentence-transformers/all-mpnet-base-v2")).chain(QAPrompt(backend.OpenAI()))
    result = prompt(question)
    print(result)

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)




Gatsby had no past before he met Daisy. He was a penniless young man without a past, and at any moment the invisible cloak of his uniform might slip from his shoulders. He was liable at the whim of an impersonal government to be blown anywhere about the world.


In [7]:
QAPrompt().show(
    {"question": "Who was Gatsby?", "docs": ["doc1", "doc2", "doc3"]}, ""
)

HTML(html='\n<!-- <link rel="stylesheet" href="https://cdn.rawgit.com/Chalarangelo/mini.css/v3.0.1/dist/mini-default.min.css"> -->\n <main class="container">\n\n<h3>QAPrompt</h3>\n\n<dl>\n  <dt>Input:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="p">{</span><span class="s1">&#39;question&#39;</span><span class="p">:</span> <span class="s1">&#39;Who was Gatsby?&#39;</span><span class="p">,</span> <span class="s1">&#39;docs&#39;</span><span class="p">:</span> <span class="p">[</span><span class="s1">&#39;doc1&#39;</span><span class="p">,</span> <span class="s1">&#39;doc2&#39;</span><span class="p">,</span> <span class="s1">&#39;doc3&#39;</span><span class="p">]}</span>\n</pre></div>\n\n\n  </dd>\n\n  <dt> Full Prompt: </dt>\n  <dd>\n    <details>\n      <summary>Prompt</summary>\n      <p>Context information is below. <br>---------------------<br><br><br>* doc1<br><br>* doc2<br><br>* doc3<br><br><br>---------------------<br><br>Given the context information and not prior knowledge, answer the question: <div style=\'color:red\'>Who was Gatsby?</div></p>\n    </details>\n  </dd>\n\n  <dt> Response: </dt>\n  <dd>\n    \n  </dd>\n\n  <dt>Value:</dt>\n  <dd>\n<div class="highlight"><pre><span></span>\n</pre></div>\n\n  </dd>\n</main>\n')

In [8]:
show_log("gatsby.log")

0bf52ad1-b1ae-4de8-b57d-9edba078ca00
└── <class '__main__.KNNPrompt'>/1 ⇒ started 2023-02-27 01:23:41Z ⧖ 0.365s
    ├── Input Function/2/1 ⇒ started 2023-02-27 01:23:41Z ⧖ 0.000s
    │   ├── input: What did Gatsby do before he met Daisy?
    │   └── Input Function/2/2 ⇒ succeeded 2023-02-27 01:23:41Z
    ├── Prompted/3/1 ⇒ started 2023-02-27 01:23:41Z ⧖ 0.363s
    │   ├── prompt: What did Gatsby do before he met Daisy?
    │   └── Prompted/3/2 ⇒ succeeded 2023-02-27 01:23:41Z
    ├── Result/4/1 ⇒ started 2023-02-27 01:23:41Z ⧖ 0.001s
    │   ├── result: 
    │   │   ├── 0: -0.003887228202074766
    │   │   ├── 1: 0.0198895875364542
    │   │   ├── 2: 0.005014391615986824
    │   │   ├── 3: -0.0190346110612154
    │   │   ├── 4: -0.042222920805215836
    │   │   ├── 5: 0.012635739520192146
    │   │   ├── 6: -0.005387490149587393
    │   │   ├── 7: 0.03442128375172615
    │   │   ├── 8: 0.0464773066341877
    │   │   ├── 9: 0.008885958231985569
    │   │   ├── 10: -0.04840860515832901
 